In [1]:
import json
import pandas as pd
import os # Wow, I'm really learning a lot about commiting, haha

from main import setup_env

setup_env()

Success: Environment variables loaded successfully.


# Getting the PUUID of summoner by name

In [2]:
import requests

def get_puuid(summoner_name, region):
    base_url = f"https://{region.lower()}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}"
    headers = {
        "X-Riot-Token": os.getenv("LEAGUE_API_KEY")
    }
    
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data.get("puuid")
    else:
        print(f"Error: {response.status_code}")
        return None

# Getting the match history of a summoner

In [3]:
def get_match_history(puuid, start_time, end_time, start, count):
    start_timestamp = date_to_timestamp(start_time)
    end_timestamp = date_to_timestamp(end_time)
    
    base_url = f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}&startTime={start_timestamp}&endTime={end_timestamp}"
    headers = {
        "X-Riot-Token": os.getenv("LEAGUE_API_KEY"),
    }
    
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        print(response.json())
        return None

# Getting the match data

In [4]:
def get_match_data(match_id):
    base_url = f"https://europe.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {
        "X-Riot-Token": os.getenv("LEAGUE_API_KEY"),
    }
    
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:   
        print(f"Error: {response.status_code}")
        print(response.json())
        return None

In [42]:
# Convert date to timestamp (seconds)
# Using timezone Europe/Bucharest
import datetime
import pytz

def date_to_timestamp(date):
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    date = pytz.timezone("Europe/Bucharest").localize(date)
    return int(date.timestamp())

def timestamp_milis_to_datetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp / 1000, pytz.timezone("Europe/Bucharest"))
    

def filter_match_data(match_data, puuid):
    filtered_data = {
        "id": match_data["metadata"]["matchId"],
        "start_time": timestamp_milis_to_datetime(match_data["info"]["gameStartTimestamp"]),
        "stop_time": timestamp_milis_to_datetime(match_data["info"]["gameEndTimestamp"]),
        "duration_in_seconds": match_data["info"]["gameDuration"],
        "duration_in_minutes": match_data["info"]["gameDuration"] / 60,
        "gamemode": match_data["info"]["gameMode"],
        "kda_formatted": None,
        "kda": None,
        "win": None,
        "champion": None,
    }
    
    for player in match_data["info"]["participants"]:
        if player["puuid"] == puuid:
            filtered_data["kda_formatted"] = f"{player['kills']}/{player['deaths']}/{player['assists']}"
            player["deaths"] = 1 if player["deaths"] == 0 else player["deaths"]
            filtered_data["kda"] = (player["kills"] + player["assists"]) / player["deaths"]
            filtered_data["win"] = player["win"]
            filtered_data["champion"] = player["championName"]
            break
            
    return filtered_data

In [44]:
def turn_data_to_dataframe(match_ids, puuid):
    matches_data = [get_match_data(match_id) for match_id in match_ids]
    filtered_matches_data = [filter_match_data(match_data, puuid) for match_data in matches_data]
    
    return pd.DataFrame(filtered_matches_data)

In [52]:
puuid = get_puuid("extremq", "EUN1")
match_history = get_match_history(puuid, "2023-08-12", "2023-08-19", 0, 100)
turn_data_to_dataframe(match_history, puuid)

,id,start_time,stop_time,duration_in_seconds,duration_in_minutes,gamemode,kda_formatted,kda,win,champion
0,EUN1_3437170687,2023-08-18 07:37:34.364000+03:00,2023-08-18 08:12:31.017000+03:00,2096,34.933333,CLASSIC,17/7/20,5.285714,True,LeeSin
1,EUN1_3436810195,2023-08-17 19:23:21.803000+03:00,2023-08-17 19:41:15.200000+03:00,1073,17.883333,ARAM,25/8/23,6.000000,True,Zed
2,EUN1_3436785545,2023-08-17 19:02:28.446000+03:00,2023-08-17 19:21:00.602000+03:00,1112,18.533333,ARAM,4/13/20,1.846154,False,Sion
3,EUN1_3436642726,2023-08-17 15:11:01.366000+03:00,2023-08-17 15:33:09.424000+03:00,1328,22.133333,ARAM,18/11/39,5.181818,True,Leona
4,EUN1_3436633168,2023-08-17 14:48:25.861000+03:00,2023-08-17 15:07:49.238000+03:00,1163,19.383333,ARAM,4/18/24,1.555556,False,Sion
5,EUN1_3436614912,2023-08-17 14:25:38.115000+03:00,2023-08-17 14:45:09.233000+03:00,1171,19.516667,ARAM,8/9/43,5.666667,True,Rell
6,EUN1_3436053023,2023-08-16 17:36:01.694000+03:00,2023-08-16 18:09:12.737000+03:00,1991,33.183333,CLASSIC,13/8/6,2.375000,False,Ezreal
7,EUN1_3436039855,2023-08-16 17:12:01.818000+03:00,2023-08-16 17:30:54.267000+03:00,1132,18.866667,CLASSIC,1/9/6,0.777778,False,Alistar
8,EUN1_3436015683,2023-08-16 16:42:24.088000+03:00,2023-08-16 17:06:18.878000+03:00,1434,23.900000,CLASSIC,6/4/12,4.500000,True,LeeSin
9,EUN1_3435999852,2023-08-16 16:06:43.076000+03:00,2023-08-16 16:35:18.626000+03:00,1715,28.583333,CLASSIC,5/8/5,1.250000,False,LeeSin


In [46]:
dragos_dataframe = _

In [49]:
rares_dataframe = _

In [53]:
stefa_dataframe = _

In [51]:
# Check how many ids are in both dataframes
len(set(dragos_dataframe["id"]) & set(rares_dataframe["id"]))

25

In [54]:
# Check how many ids are in both dataframes
len(set(dragos_dataframe["id"]) & set(stefa_dataframe["id"]))

13

In [55]:
# Check how many ids are in all dataframes
len(set(dragos_dataframe["id"]) & set(rares_dataframe["id"]) & set(stefa_dataframe["id"]))

13

In [56]:
# Check how many ids are in both
len(set(stefa_dataframe["id"]) & set(rares_dataframe["id"]))

19